In [23]:
import pandas as pd

In [24]:
df = pd.read_excel('../data/returns_RU.xlsx', parse_dates=[0])
df.head()

,Date,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,BANEP,...,TATNP,TCSG,TGKA,TRNFP,TTLK,UPRO,VKCO,VSMO,VTBR,YNDX
0,2018-01-03,12.350,140.4,570.0,3930.0,75.81,4.82,8.65,140.5,1357.0,...,367.7,NaN,0.012185,179300,0.1445,2.567,NaN,17000.0,0.04950,1943.5
1,2018-01-04,12.385,148.9,565.0,3994.0,77.00,4.75,8.59,152.0,1364.5,...,368.7,NaN,0.012285,182900,0.1450,2.554,NaN,17250.0,0.05005,1978.0
2,2018-01-05,12.390,147.3,562.5,3985.0,76.90,4.77,8.59,155.0,1368.5,...,371.8,NaN,0.012355,182500,0.1480,2.568,NaN,17300.0,0.05020,1942.5
3,2018-01-09,12.715,150.5,566.5,3982.0,76.79,4.80,8.50,162.0,1390.0,...,380.2,NaN,0.012480,179500,0.1500,2.600,NaN,17430.0,0.05080,1963.0
4,2018-01-10,12.700,149.3,570.0,3995.0,76.70,4.88,8.40,160.0,1444.0,...,382.0,NaN,0.012530,178750,0.1530,2.594,NaN,17450.0,0.05051,1972.0


In [25]:
df['Weekday'] = df['Date'].dt.dayofweek

In [26]:
df.head()

,Date,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,BANEP,...,TCSG,TGKA,TRNFP,TTLK,UPRO,VKCO,VSMO,VTBR,YNDX,Weekday
0,2018-01-03,12.350,140.4,570.0,3930.0,75.81,4.82,8.65,140.5,1357.0,...,NaN,0.012185,179300,0.1445,2.567,NaN,17000.0,0.04950,1943.5,2
1,2018-01-04,12.385,148.9,565.0,3994.0,77.00,4.75,8.59,152.0,1364.5,...,NaN,0.012285,182900,0.1450,2.554,NaN,17250.0,0.05005,1978.0,3
2,2018-01-05,12.390,147.3,562.5,3985.0,76.90,4.77,8.59,155.0,1368.5,...,NaN,0.012355,182500,0.1480,2.568,NaN,17300.0,0.05020,1942.5,4
3,2018-01-09,12.715,150.5,566.5,3982.0,76.79,4.80,8.50,162.0,1390.0,...,NaN,0.012480,179500,0.1500,2.600,NaN,17430.0,0.05080,1963.0,1
4,2018-01-10,12.700,149.3,570.0,3995.0,76.70,4.88,8.40,160.0,1444.0,...,NaN,0.012530,178750,0.1530,2.594,NaN,17450.0,0.05051,1972.0,2


In [27]:
for col in df.columns[1:-1]:
    df[col+'_pct'] = df[col].pct_change()
df.head()

C:\Users\SNIPER~1\AppData\Local\Temp/ipykernel_7176/1208546071.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_pct'] = df[col].pct_change()


,Date,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,BANEP,...,TATNP_pct,TCSG_pct,TGKA_pct,TRNFP_pct,TTLK_pct,UPRO_pct,VKCO_pct,VSMO_pct,VTBR_pct,YNDX_pct
0,2018-01-03,12.350,140.4,570.0,3930.0,75.81,4.82,8.65,140.5,1357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-04,12.385,148.9,565.0,3994.0,77.00,4.75,8.59,152.0,1364.5,...,0.002720,NaN,0.008207,0.020078,0.003460,-0.005064,NaN,0.014706,0.011111,0.017751
2,2018-01-05,12.390,147.3,562.5,3985.0,76.90,4.77,8.59,155.0,1368.5,...,0.008408,NaN,0.005698,-0.002187,0.020690,0.005482,NaN,0.002899,0.002997,-0.017947
3,2018-01-09,12.715,150.5,566.5,3982.0,76.79,4.80,8.50,162.0,1390.0,...,0.022593,NaN,0.010117,-0.016438,0.013514,0.012461,NaN,0.007514,0.011952,0.010553
4,2018-01-10,12.700,149.3,570.0,3995.0,76.70,4.88,8.40,160.0,1444.0,...,0.004734,NaN,0.004006,-0.004178,0.020000,-0.002308,NaN,0.001147,-0.005709,0.004585


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Columns: 200 entries, Date to YNDX_pct
dtypes: datetime64[ns](1), float64(195), int64(4)
memory usage: 1.8 MB


In [30]:
from utils.prepare_data import prepare_short_data

In [31]:
prepare_short_data('data/returns_RU.xlsx')

C:\Users\snipercapt\Desktop\PAAI\final\utils\prepare_data.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col+'_pct'] = df[col].pct_change()


,Date,AFKS,AFLT,AGRO,AKRN,ALRS,AMEZ,APTK,AQUA,BANEP,...,TATNP_pct,TCSG_pct,TGKA_pct,TRNFP_pct,TTLK_pct,UPRO_pct,VKCO_pct,VSMO_pct,VTBR_pct,YNDX_pct
0,2018-01-03,12.350,140.40,570.0,3930.0,75.81,4.820,8.650,140.5,1357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-04,12.385,148.90,565.0,3994.0,77.00,4.750,8.590,152.0,1364.5,...,0.002720,NaN,0.008207,0.020078,0.003460,-0.005064,NaN,0.014706,0.011111,0.017751
2,2018-01-05,12.390,147.30,562.5,3985.0,76.90,4.770,8.590,155.0,1368.5,...,0.008408,NaN,0.005698,-0.002187,0.020690,0.005482,NaN,0.002899,0.002997,-0.017947
3,2018-01-09,12.715,150.50,566.5,3982.0,76.79,4.800,8.500,162.0,1390.0,...,0.022593,NaN,0.010117,-0.016438,0.013514,0.012461,NaN,0.007514,0.011952,0.010553
4,2018-01-10,12.700,149.30,570.0,3995.0,76.70,4.880,8.400,160.0,1444.0,...,0.004734,NaN,0.004006,-0.004178,0.020000,-0.002308,NaN,0.001147,-0.005709,0.004585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,2022-10-25,12.321,25.16,825.8,15296.0,66.96,26.050,12.202,538.0,634.0,...,0.027011,0.021484,0.006474,0.031095,0.004301,0.058591,0.024223,-0.001437,0.029229,0.023715
1198,2022-10-26,12.649,25.10,829.0,15450.0,66.36,26.055,12.064,558.5,631.0,...,0.003637,-0.007937,-0.010391,0.009650,0.002141,0.030666,0.004462,0.004317,0.012387,-0.006039
1199,2022-10-27,12.770,25.56,842.8,15688.0,67.03,26.460,12.230,562.5,650.0,...,0.007806,-0.001200,0.002500,0.064516,0.002137,0.002177,-0.008885,-0.005731,0.004775,0.006474
1200,2022-10-28,12.696,25.52,833.0,16110.0,66.40,26.515,12.334,582.0,657.5,...,0.005533,0.002403,-0.003242,0.045455,0.001066,0.008689,0.000448,-0.003242,0.004752,0.000396
